Importing modules

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score
from Layer import *
from network import Network
from activation_func import tanh, tanh_prime, sigmoid, sigmoid_prime, softmax, softmax_prime, relu, relu_prime
from loss_func import mse, mse_prime, cross_entropy, cross_entropy_prime
from data_func import vectorize_labels, k_fold, import_data
from performance_func import plot_error, plot_confusion_matrix
from network_queue import Queue


Importing data

In [2]:

# import data and initialize seed
np.random.seed(10)

# import data
training_size = 100
normalize = True
training, labels, test, original_test_labels, test_labels = import_data(size=training_size, normalize=normalize)

# intializing queue
queue = Queue(training, labels, test, test_labels, original_test_labels)

#setting up the models

architectures = [[15]]
params =  {"epochs": 3,
            "learning_rate": 5e-3,
            "batch_size": 1,
            "momentum": False,
            "weight_decay": 0.01}

for layers in architectures:
    queue.add(
        description="relu - activation benchmark",
        netparams={
        "hidden_layers": layers,
        "features": 784,
        "output_classes": 10,
        "activation": relu,
        "activation_prime": relu_prime,
        "loss_activation": softmax,
        "loss_activation_prime": softmax_prime,
        "loss": cross_entropy,
        "loss_prime": cross_entropy_prime,
        "activation_alpha": 0.5
    },
    folds=5,
    params=params)
    queue.add(
        description="sigmoid - activation benchmark",
        netparams={
        "hidden_layers": layers,
        "features": 784,
        "output_classes": 10,
        "activation": sigmoid,
        "activation_prime": sigmoid_prime,
        "loss_activation": softmax,
        "loss_activation_prime": softmax_prime,
        "loss": cross_entropy,
        "loss_prime": cross_entropy_prime,
        "activation_alpha": 0.5
    },
    folds=5,
    params=params)

#execute queue
results = queue.execute()

Fitting model 1/10
epoch 1/3   training error=4.504946  validation error=3.579496 validation accuracy=0.000000 ETA=0:00:00
epoch 2/3   training error=2.654058  validation error=2.936414 validation accuracy=0.100000 ETA=0:00:00
epoch 3/3   training error=2.033245  validation error=2.673691 validation accuracy=0.150000 ETA=0:00:00
Average epoch computational time:  0.013322591781616211
Fitting model 2/10
epoch 1/3   training error=4.883202  validation error=3.162335 validation accuracy=0.300000 ETA=0:00:00
epoch 2/3   training error=2.761088  validation error=2.447065 validation accuracy=0.350000 ETA=0:00:00
epoch 3/3   training error=2.039783  validation error=2.280197 validation accuracy=0.350000 ETA=0:00:00
Average epoch computational time:  0.011930068333943685
Fitting model 3/10
epoch 1/3   training error=4.093267  validation error=4.204655 validation accuracy=0.200000 ETA=0:00:00
epoch 2/3   training error=2.356893  validation error=3.569386 validation accuracy=0.250000 ETA=0:00:00

TypeError: can only concatenate str (not "float") to str

In [9]:
model_1 = np.load("Results/architecture [50, 30] training_size 60000 - 2019-04-03-135327.npy")

print(model_1[1][])

{'description': 'architecture: [50, 30] training_size: 60000 - Fold 2', 'network': <network.Network object at 0x0000026FA13010F0>, 'params': {'epochs': 20, 'learning_rate': 0.005, 'batch_size': 1, 'momentum': False, 'weight_decay': 0.01}, 'results': [[1.0601139293489, 0.817552704042831, 0.8006510164544239, 0.78864133180134, 0.7822959641275584, 0.777948132629834, 0.7741978886001222, 0.7719888599166445, 0.7683601891819973, 0.7667327598248964, 0.7647189040804909, 0.7644301190706472, 0.7642026596477887, 0.761641912618608, 0.7632715036641381, 0.7626014319699548, 0.7615784210946479, 0.7613994081882737, 0.7604580027128008, 0.7599084114413943], [0.8478366281433534, 0.8231635036066182, 0.8051249552319448, 0.8013794597581395, 0.7976891430454346, 0.7898043897293414, 0.7855253617817358, 0.7859953331665902, 0.7736320206578458, 0.7749338274786193, 0.7880536642516568, 0.7756203856388009, 0.7750925691413421, 0.779924228616348, 0.7723104165972298, 0.7670183536374792, 0.7710343698904785, 0.7720523551937